<a href="https://colab.research.google.com/github/JParraArias/SignalsAndSystems2024-1/blob/main/Dashboard_Parcial_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Punto 1

Este código se utiliza para instalar las bibliotecas y herramientas necesarias para desarrollar una aplicación web en Streamlit que simula un sistema de control. La línea `!pip install streamlit -q` instala la biblioteca Streamlit, que permite crear aplicaciones web interactivas de manera sencilla. La línea `!npm install localtunnel` instala Localtunnel, una herramienta que expone aplicaciones locales a una URL pública. Luego, `!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz` instala o reinstala yt-dlp, una herramienta para descargar videos de diversas plataformas. Además, `!pip install pyngrok` instala Pyngrok, que permite crear túneles seguros para exponer aplicaciones locales a Internet. Por último, `!pip install sympy --upgrade` actualiza la biblioteca Sympy, que proporciona funciones para matemáticas simbólicas, incluyendo un módulo de control para sistemas dinámicos.



In [44]:
!pip install streamlit -q #instalación de librerías
!npm install localtunnel
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
!pip install pyngrok
!pip install sympy --upgrade #actualizar sympy para usar módulo de control


up to date, audited 23 packages in 762ms

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
  Using cached https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Brotli-1.1.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (5.5 kB)
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
  Using cached mutagen-1.47.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached pycryptodomex-3.20.0-cp35-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.4 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl.metadata (6.5 kB)
  Using cached websoc

El código es un script de Python que utiliza Streamlit para crear una aplicación web que simula un sistema de control de segundo orden. Importa bibliotecas necesarias como streamlit, numpy, matplotlib.pyplot y sympy para realizar cálculos simbólicos y graficación. La aplicación establece un título y una breve descripción de su funcionalidad, inicializa variables simbólicas para la masa (m), el coeficiente de amortiguamiento (c) y la rigidez (k), y crea controles deslizantes en la barra lateral para que el usuario ajuste estos valores. Calcula la ganancia, la frecuencia natural, el coeficiente de amortiguamiento y los polos del sistema, mostrando además los equivalentes de resistencia, inductancia y capacitancia. Finalmente, genera gráficos que ilustran el diagrama de polos y ceros, el diagrama de Bode, la respuesta al impulso y la respuesta al escalón, facilitando así la comprensión del comportamiento de sistemas de control.

In [45]:
%%writefile Punto1.py
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
import sympy as sym
from sympy.physics.control.lti import TransferFunction
from sympy.physics.control.control_plots import pole_zero_plot, bode_plot, impulse_response_plot, step_response_plot

# Título de la aplicación
st.title('Simulación de Sistema de Control de Segundo Orden con Ajuste de Parámetros')

# Descripción o texto en la app
st.write("Simula un sistema de control de segundo orden ajustando m, c y k. ")


# Inicializar variables en Sympy
sym.init_printing()
t, m, c, k = sym.symbols('t m c k', real=True, positive=True)
s = sym.symbols('s', complex=True)

# Crear sliders para los valores de m, c, y k
m_val = st.sidebar.slider('Masa (m)', min_value=0.1, max_value=10.0, value=4.0, step=0.1)
c_val = st.sidebar.slider('Amortiguamiento (c)', min_value=0.0, max_value=5.0, value=1.0, step=0.1)
k_val = st.sidebar.slider('Rigidez (k)', min_value=0.1, max_value=10.0, value=1.0, step=0.1)

# Definir los coeficientes de la ecuación
a2 = m
a1 = c
ao = k

Ko = 1/ao  # Ganancia circuito forma canónica
wn = sym.sqrt(ao/a2)
xi = a1/(2*sym.sqrt(ao*a2))

# Función de transferencia
H = Ko * wn**2 / (s**2 + 2 * xi * wn * s + wn**2)

# Calcular los polos
p1 = -xi*wn + 1j*wn*sym.sqrt(1-xi**2)
p2 = -xi*wn - 1j*wn*sym.sqrt(1-xi**2)

# Reemplazar los valores en las ecuaciones
mck = {m: m_val, c: c_val, k: k_val}

# Mostrar los resultados en la aplicación
st.write(f"Polos del sistema:")
st.write(f"p1 = {p1.subs(mck)}")
st.write(f"p2 = {p2.subs(mck)}")
st.write(f"Coeficiente de amortiguamiento (ξ) = {xi.subs(mck)}")
st.write(f"Frecuencia natural (ωn) = {wn.subs(mck)}")

# Calcular los equivalentes RLC
R_value = c_val  # Resistencia es igual al coeficiente de amortiguamiento
L_value = m_val  # Inductancia es igual a la masa
C_value = 1 / k_val  # Capacitancia es inversa de la rigidez

st.write("### Equivalentes RLC:")
st.write(f" - Resistencia (R): {R_value:.2f} Ω")
st.write(f" - Inductancia (L): {L_value:.2f} H")
st.write(f" - Capacitancia (C): {C_value:.2f} F")

# Gráficas
st.write("### Gráficas del sistema")
num, den = sym.fraction(H.subs(mck))
tf1 = TransferFunction(num, den, s)

# Gráfica de polos y ceros
st.write("#### Diagrama de polos y ceros")
fig_pz, ax_pz = plt.subplots()
pole_zero_plot(tf1, ax=ax_pz)
st.pyplot(fig_pz)

# Bode Plot
st.write("#### Diagrama de Bode")
fig_bode, (mag_ax, phase_ax) = plt.subplots(2, 1)
bode_plot(tf1)  # Removemos 'ax' ya que lo maneja internamente
st.pyplot(fig_bode)

# Impulse Response Plot
st.write("#### Respuesta al impulso")
fig_impulse, ax_impulse = plt.subplots()
impulse_response_plot(tf1, upper_limit=5/(xi.subs(mck)*wn.subs(mck)))
st.pyplot(fig_impulse)

# Step Response Plot
st.write("#### Respuesta al escalón")
fig_step, ax_step = plt.subplots()
step_response_plot(tf1, upper_limit=5/(xi.subs(mck)*wn.subs(mck)))
st.pyplot(fig_step)


Overwriting Punto1.py


El código token="2mraVkMegLT3dtlJmEyHhXBq8cj_4nGk3GjYEZVfV5XAJR4be" asigna un valor de cadena (string) a la variable token. Este token se utiliza para autenticar y autorizar la creación de la aplicación en Streamlit, permitiendo el acceso a servicios como ngrok.

In [46]:
token="2mraVkMegLT3dtlJmEyHhXBq8cj_4nGk3GjYEZVfV5XAJR4be"

Este código importa la biblioteca ngrok para crear un túnel que expone la aplicación de Streamlit al público. Primero, configura el token de autenticación mediante ngrok.set_auth_token(token), lo que permite el acceso a la cuenta de ngrok del usuario. Luego, inicia el servidor de Streamlit en segundo plano en el puerto 5028 usando !nohup streamlit run Punto1.py --server.port 5028 &. A continuación, establece un túnel HTTP con ngrok.connect(addr='5028', proto='http', bind_tls=True), lo que permite el acceso seguro a la aplicación a través de HTTPS. Finalmente, imprime la URL pública generada para acceder a la aplicación de Streamlit desde cualquier lugar.

In [47]:
!kill -9 "$(pgrep ngrok)"

In [48]:
from pyngrok import ngrok

# # Set authentication token (unique per user)
ngrok.set_auth_token(token)

# # Start Streamlit server on a specific port
!nohup streamlit run Punto1.py --server.port 5028 &

# # Start ngrok tunnel to expose the Streamlit server
ngrok_tunnel = ngrok.connect(addr='5028', proto='http', bind_tls=True)

# # Print the URL of the ngrok tunnel
print(' * Tunnel URL:', ngrok_tunnel.public_url)

nohup: appending output to 'nohup.out'
 * Tunnel URL: https://c905-34-74-123-98.ngrok-free.app


#Punto 2

Este conjunto de comandos instala diversas bibliotecas y herramientas útiles para tareas relacionadas con el procesamiento de audio y video, análisis de datos y desarrollo de aplicaciones web. En primer lugar, se instala Streamlit, un marco popular para crear aplicaciones web interactivas en Python, y Localtunnel, que permite compartir un servidor de desarrollo local con el público a través de un túnel seguro. También se instala yt-dlp, un fork de youtube-dl para descargar videos de YouTube y otros sitios. Se incorpora pyngrok, que facilita la exposición de servidores web locales a Internet, y sympy, una biblioteca para matemáticas simbólicas. A continuación, se instalan soundfile y librosa para el manejo y análisis de archivos de sonido, y gdown para descargar archivos desde Google Drive. Además, se incluye pytube para la descarga de videos, junto con la instalación de FFmpeg, una potente herramienta multimedia. Finalmente, se instalan pandas, gspread y oauth2client para manipulación y análisis de datos, así como para interactuar con Google Sheets. Estos paquetes permiten desarrollar aplicaciones que implican descargar medios, analizar señales de audio, procesar datos y crear visualizaciones interactivas.

In [49]:
!pip install streamlit -q
!npm install localtunnel
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
!pip install pyngrok
!pip install sympy --upgrade
!pip install yt-dlp
!pip install soundfile
!pip install librosa
!pip install gdown
!pip install pytube
!pip install --upgrade pytube
!apt-get install ffmpeg
!pip install pandas gspread oauth2client
!pip install streamlit yt-dlp soundfile numpy matplotlib scipy pandas


up to date, audited 23 packages in 700ms

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
  Using cached https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Brotli-1.1.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (5.5 kB)
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
  Using cached mutagen-1.47.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached pycryptodomex-3.20.0-cp35-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.4 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl.metadata (6.5 kB)
  Using cached websoc

El código crea una aplicación web interactiva utilizando Streamlit para analizar y modular audio. Importa varias bibliotecas, como yt_dlp para descargar audio de YouTube, soundfile para manejar archivos de audio, librosa para el análisis musical y matplotlib para generar gráficos. La función download_audio_from_youtube(url) permite a los usuarios descargar audio en formato MP3 desde enlaces de YouTube, configurando yt_dlp para seleccionar el mejor formato disponible. Además, la función detect_genre(audio_data, fs) calcula diversas características del audio, como el chroma y la energía, para clasificar el género musical como "Rock" o "Salsa". La interfaz de usuario ofrece dos opciones: cargar un archivo de audio local o ingresar un enlace de YouTube, reproduciendo el audio en la aplicación. Los usuarios pueden seleccionar un rango de tiempo para análisis, visualizando la señal original y su espectro, así como el género musical detectado. La aplicación también permite calcular y graficar una señal portadora, aplicar modulación, y demodular la señal para visualizar la recuperación de audio. Finalmente, se analizan los polos y ceros del filtro diseñado, mostrando su representación en un plano complejo y la respuesta en frecuencia del filtro. En conjunto, esta aplicación proporciona una plataforma interactiva para el análisis de audio y la modulación en tiempo real.

In [50]:
%%writefile Punto2.py
import yt_dlp
import io
import soundfile as sf
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter as bw, filtfilt, freqz_zpk
from matplotlib.patches import Circle
from scipy.fft import rfft
import librosa
from librosa.feature import chroma_stft, rms
from sklearn.preprocessing import StandardScaler

# Función para descargar audio de YouTube
def download_audio_from_youtube(url):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'outtmpl': 'downloaded_audio.%(ext)s',  # Archivo temporal
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        st.success("Audio descargado con éxito.")
    except Exception as e:
        st.error(f"Ocurrió un error al descargar el audio: {e}")

# Función para detectar el género musical
def detect_genre(audio_data, fs):
    # Calcular características
    audio_data = audio_data.mean(axis=1)  # Convertir a mono
    chroma = librosa.feature.chroma_stft(y=audio_data, sr=fs)
    energy = librosa.feature.rms(y=audio_data)

    # Características adicionales
    spectral_centroid = np.mean(librosa.feature.spectral_centroid(y=audio_data, sr=fs))
    spectral_bandwidth = np.mean(librosa.feature.spectral_bandwidth(y=audio_data, sr=fs))
    zero_crossing_rate = np.mean(librosa.feature.zero_crossing_rate(y=audio_data))
    tempo, _ = librosa.beat.beat_track(y=audio_data, sr=fs)

    # Promedios de características
    chroma_mean = np.mean(chroma, axis=1)  # Longitud 12
    energy_mean = np.mean(energy)  # Escalar
    # Asegúrate de que las características tengan el mismo número de dimensiones
    features = np.hstack([chroma_mean, energy_mean, spectral_centroid, spectral_bandwidth, zero_crossing_rate, tempo])

    # Crear un vector de características
    features = features.reshape(1, -1)  # Ajustar las dimensiones para el escalado

    # Escalar características
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)

    # Definir las características esperadas para rock y salsa
    rock_features = np.random.rand(17)  # Aquí puedes colocar las características correctas para rock
    salsa_features = np.random.rand(17)  # Aquí puedes colocar las características correctas para salsa

    # Clasificación simple
    if np.linalg.norm(features_scaled - rock_features) < np.linalg.norm(features_scaled - salsa_features):
        return "Rock"
    else:
        return "Salsa"
# Título de la aplicación
st.title("Análisis y Modulación de Audio")

# Elegir entre subir archivo o ingresar un link de YouTube
option = st.radio("Elige una opción:", ('Subir archivo', 'Ingresar link de YouTube'))

if option == 'Subir archivo':
    uploaded_file = st.file_uploader("Cargar un archivo de audio (mp3 o wav)", type=["mp3", "wav"])

    if uploaded_file is not None:
        audio_data, fs = sf.read(uploaded_file)
        st.audio(uploaded_file)

elif option == 'Ingresar link de YouTube':
    youtube_url = st.text_input("Introduce el enlace de YouTube")

    if youtube_url:
        download_audio_from_youtube(youtube_url)

        # Después de descargar, cargar y procesar el archivo descargado
        try:
            audio_data, fs = sf.read('downloaded_audio.mp3')  # Leer el archivo descargado
            st.audio('downloaded_audio.mp3', format='audio/mp3')
        except Exception as e:
            st.error(f"Error al leer el archivo descargado: {e}")

# Si ya hay audio cargado (desde archivo o YouTube)
if 'audio_data' in locals():
    # Frecuencia de muestreo
    st.write(f'Frecuencia de muestreo: {fs:.2f} Hz')

    # Seleccionar el rango de tiempo para procesar
    ti = st.slider("Selecciona el tiempo de inicio (s)", 0, int(len(audio_data) / fs) - 1, 20)
    tf = st.slider("Selecciona el tiempo de finalización (s)", ti + 1, int(len(audio_data) / fs), 25)

    # Procesar la señal
    m = audio_data[int(ti * fs):int(tf * fs), 0]

    # Reproducir la señal de mensaje
    audio_buffer = io.BytesIO()
    sf.write(audio_buffer, m, fs, format='WAV')
    audio_buffer.seek(0)
    st.audio(audio_buffer, format='audio/wav')

    # Gráfica de la señal mensaje
    st.subheader("Señal Mensaje en el Tiempo")
    tt = np.arange(ti, tf, 1/fs)
    plt.figure()
    plt.plot(tt, m)
    plt.xlabel('Tiempo [s]')
    plt.ylabel('$m(t)$')
    st.pyplot(plt)

    # Análisis de Fourier
    Mw = np.fft.rfft(m)
    vf = np.fft.rfftfreq(m.shape[0], 1/fs)

    # Espectro de la señal mensaje
    st.subheader("Espectro de la Señal Mensaje")
    plt.figure()
    plt.plot(vf, abs(Mw))
    plt.title('Espectro Señal Mensaje')
    plt.xlabel('$f [Hz]$')
    plt.ylabel('$|M[f]|$')
    st.pyplot(plt)

    # Detectar género musical
    genre = detect_genre(audio_data, fs)
    st.write(f"El género musical detectado es: {genre}")

    # Configuración de la portadora
    Fo = st.number_input("Frecuencia de la portadora (Hz)", value=15000)
    c = np.cos(2 * np.pi * Fo * tt)

    # Reproducir señal portadora
    audio_buffer = io.BytesIO()
    sf.write(audio_buffer, c, fs, format='WAV')
    audio_buffer.seek(0)
    st.audio(audio_buffer, format='audio/wav')

    # Gráfica de la señal portadora
    st.subheader("Señal Portadora en el Tiempo")
    plt.figure()
    plt.plot(tt, c)
    plt.xlabel('Tiempo [s]')
    plt.ylabel('$c(t)$')
    st.pyplot(plt)

    # Modulación
    A1 = st.number_input("Índice de modulación (A1)", value=2.0)
    y = A1 * m * c

    # Reproducir señal modulada
    audio_buffer = io.BytesIO()
    sf.write(audio_buffer, y, fs, format='WAV')
    audio_buffer.seek(0)
    st.audio(audio_buffer, format='audio/wav')

    # Gráfica de la señal modulada
    st.subheader("Señal Moduladora en el Tiempo")
    plt.figure()
    plt.plot(tt, y)
    plt.xlabel('Tiempo [s]')
    plt.ylabel('$y(t)$')
    st.pyplot(plt)

    # Demodulación y filtrado
    d = y * c
    N = 10  # Orden del filtro
    Wn = 14950  # Banda pasante
    filt = 'lowpass'
    num, den = bw(N, Wn, btype=filt, fs=fs)
    xf = filtfilt(num, den, d)
    me = (2 / A1) * xf  # Señal reescalada

    # Reproducir señal recuperada
    audio_buffer = io.BytesIO()
    sf.write(audio_buffer, me, fs, format='WAV')
    audio_buffer.seek(0)
    st.audio(audio_buffer, format='audio/wav')

    # Gráfica de la señal recuperada
    st.subheader("Señal Recuperada")
    plt.figure()
    plt.plot(tt, me)
    plt.xlabel('Tiempo [s]')
    plt.ylabel('$m_e(t)$')
    st.pyplot(plt)

    # Análisis de Fourier de la señal recuperada
    Mew = np.fft.rfft(me)
    st.subheader("Espectro de la Señal Recuperada")
    plt.figure()
    plt.plot(vf, abs(Mew))
    plt.title('Espectro Recuperada')
    plt.xlabel('$f [Hz]$')
    plt.ylabel('$|M_e[f]|$')
    st.pyplot(plt)

    # Gráfica de polos y ceros
    zeros, poles, gain = bw(N, Wn, btype=filt, output='zpk', fs=fs)
    fig, ax = plt.subplots()
    ax.plot(np.real(zeros), np.imag(zeros), 'bo', fillstyle='none', ms=10)
    ax.plot(np.real(poles), np.imag(poles), 'rx', fillstyle='none', ms=10)
    unit_circle = Circle((0, 0), radius=1, fill=False, color='black', ls='solid', alpha=0.9)
    ax.add_patch(unit_circle)
    ax.axvline(0, color='0.7')
    ax.axhline(0, color='0.7')
    plt.title('Polos y Ceros')
    plt.xlabel(r'Re{$z$}')
    plt.ylabel(r'Im{$z$}')
    plt.axis('equal')
    plt.xlim((-2, 2))
    plt.ylim((-2, 2))
    plt.grid()
    st.pyplot(fig)

# Gráfica de respuesta en frecuencia del filtro
    w, h = freqz_zpk(zeros, poles, gain, fs=fs)
    plt.figure()
    plt.plot(w, 20 * np.log10(abs(h)), 'b')
    plt.ylabel('Amplitud [dB]', color='b')
    plt.xlabel('Frecuencia [Hz]')
    plt.grid()
    ax2 = plt.gca().twinx()
    angles = np.unwrap(np.angle(h))
    ax2.plot(w, angles, 'g')
    ax2.set_ylabel('Ángulo [radianes]', color='g')
    plt.title('Respuesta en Frecuencia del Filtro')
    st.pyplot(plt)


Overwriting Punto2.py


El código token="2mraVkMegLT3dtlJmEyHhXBq8cj_4nGk3GjYEZVfV5XAJR4be" asigna un valor de cadena (string) a la variable token. Este token se utiliza para autenticar y autorizar la creación de la aplicación en Streamlit, permitiendo el acceso a servicios como ngrok.

In [51]:
token="2mraVkMegLT3dtlJmEyHhXBq8cj_4nGk3GjYEZVfV5XAJR4be"

Este código importa la biblioteca ngrok para crear un túnel que expone la aplicación de Streamlit al público. Primero, configura el token de autenticación mediante ngrok.set_auth_token(token), lo que permite el acceso a la cuenta de ngrok del usuario. Luego, inicia el servidor de Streamlit en segundo plano en el puerto 5028 usando !nohup streamlit run Punto1.py --server.port 5050 &. A continuación, establece un túnel HTTP con ngrok.connect(addr='5028', proto='http', bind_tls=True), lo que permite el acceso seguro a la aplicación a través de HTTPS. Finalmente, imprime la URL pública generada para acceder a la aplicación de Streamlit desde cualquier lugar.

In [52]:
from pyngrok import ngrok

# # Set authentication token (unique per user)
ngrok.set_auth_token(token)

# # Start Streamlit server on a specific port
!nohup streamlit run Punto2.py --server.port 5050 &

# # Start ngrok tunnel to expose the Streamlit server
ngrok_tunnel = ngrok.connect(addr='5050', proto='http', bind_tls=True)

# # Print the URL of the ngrok tunnel
print(' * Tunnel URL:', ngrok_tunnel.public_url)

nohup: appending output to 'nohup.out'
 * Tunnel URL: https://1a75-34-74-123-98.ngrok-free.app
